In [31]:
# Setup the environment
!pip install --upgrade huggingface_hub >> /dev/null
!pip install git+https://github.com/huggingface/transformers -U accelerate peft >> /dev/null
!pip install -i https://pypi.org/simple/ bitsandbytes >> /dev/null

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-ns1b7f8s


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [32]:
import transformers
transformers.__version__

'4.39.0.dev0'

In [33]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)
import torch


In [34]:
# Load the model
model_id = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [35]:
model = AutoModelForCausalLM.from_pretrained(model_id,
                                            device_map="auto")

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [ ]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
def test_prompt(prompt, your_model):
    input_ids = tokenizer(prompt, return_tensors='pt')
    logits = your_model.generate(**input_ids.to(device),
                           max_new_tokens=50)
    return tokenizer.decode(logits[0])

In [ ]:
tokenizer.max_len_single_sentence

In [ ]:
text_test = "Give me the top 15 golf equipment company names."

orginal_model_out = test_prompt(text_test, model)

orginal_model_out

In [ ]:
from transformers import (
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from datasets import load_dataset, load_metric, Dataset
import pandas as pd

In [ ]:
import json
data = []

with open('/kaggle/input/databricks-dolly-15k/databricks-dolly-15k.jsonl') as file:
    for line in file:
        features = json.loads(line)
        # Filter out examples with context, to keep it simple.
        if features["context"]:
            continue
        # Format the entire example as a single string.
        template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
        data.append(template.format(**features))

In [ ]:
data_500 = [x for x in data if len(x) <= 500]

In [ ]:
len(data_500)

In [ ]:
data_500[10:25]

In [ ]:
# take 1500 from that data 
data = data_500[:2500]

In [ ]:
tokenizer(data[0])

In [ ]:
json_ds = Dataset.from_pandas(pd.DataFrame(data=data))

In [ ]:
def tokenize_instruction(row):
    # print(row)
    input_token = tokenizer(row['0'], padding="max_length",
                            truncation=True, max_length=512)
    input_token['labels'] = input_token['input_ids'].copy()
    return input_token

In [ ]:
dataset_tokenized = json_ds.map(tokenize_instruction,
                                remove_columns=["0"],
                                batched=True)

In [ ]:
process_ds = dataset_tokenized.train_test_split(test_size=0.2)

In [ ]:
process_ds

In [ ]:
len(process_ds['train'][1850]['input_ids'])

In [ ]:
model.get_memory_footprint() / (1024 * 1024 * 1024)

In [ ]:
steps_per_epoch=len(process_ds["train"])//(8*1)

steps_per_epoch

In [ ]:
args = TrainingArguments(
    output_dir="gpt_qlora_minimal",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="steps",
    logging_steps=1,
    eval_steps=steps_per_epoch,		# eval and save once per epoch  	
    save_strategy="epoch",
    num_train_epochs=5,
    lr_scheduler_type="constant",
    # optim="paged_adamw_32bit",
    learning_rate=0.0002,
    group_by_length=True,
    # fp16=True,
    # ddp_find_unused_parameters=False,
    push_to_hub=False,
    report_to="none"
)

In [ ]:
datacollator = DataCollatorForLanguageModeling(tokenizer=tokenizer,
                                              mlm=False)

In [ ]:
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=datacollator,
    train_dataset=process_ds["train"],
    eval_dataset=process_ds["test"],
    args=args,
)

In [ ]:
# process_ds['train'][0]

In [ ]:
trainer.train()

In [ ]:
text_test = "What is Sunshine Recession?"

orginal_model_out = test_prompt(text_test, model)

trained_model_out = test_prompt(text_test, merged_model)

print("Base model")
print(original_model_out)

print("FT model")
print(trained_model_outed)